Deep Learning
=============

Assignment 4 | Tianzi Cai | 2016-07-02

------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
try:
  import tensorflow as tf
  print("TensorFlow is already installed")
except ImportError:
  print("Installing TensorFlow")
  import subprocess
  subprocess.check_call(["/databricks/python/bin/pip", "install", "https://storage.googleapis.com/tensorflow/linux/cpu/tensorflow-0.6.0-cp27-none-linux_x86_64.whl"])
  print("TensorFlow has been installed on this cluster")

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [5]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [7]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

In [8]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [10]:
'''
tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)

Computes a 2-D convolution given 4-D input and filter tensors.

Given an input tensor of shape `[batch, in_height, in_width, in_channels]` and a filter / kernel tensor of shape `[filter_height, filter_width, in_channels, out_channels]`, this op performs the following:

1. Flattens the filter to a 2-D matrix with shape `[filter_height * filter_width * in_channels, output_channels]`.
2. Extracts image patches from the input tensor to form a virtual tensor of shape `[batch, out_height, out_width, filter_height * filter_width * in_channels]`.
3. For each patch, right-multiplies the filter matrix and the image patch vector.


'''

In [11]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels)) #num_channels = 1 is grayscale
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1)) # for use in the fully connected layer
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, strides = [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset) # [16,10]
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [12]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
      
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

In [13]:
batch_size = 16 # equivalent to training 16 feature maps at a time, each of size 28*28
patch_size = 5 # kernal size of 5*5
depth = 16 # equivalent to out_channel for filter in tf.nn.conv2d(), how do we pick out_channel? It should be roughly the same size as the final out_height*out_weight...
num_hidden = 64 # num of hidden nodes in the fully connected layer

# Variables.
layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], #[filter_height, filter_width, in_channels, out_channels]
                                                 stddev=0.1)) # Why is stddev = 0.1?
layer1_biases = tf.Variable(tf.zeros([depth]))

# Convolution starts.
conv = tf.nn.conv2d(train_dataset[0:16,], layer1_weights, strides = [1, 2, 2, 1], padding='SAME') # SAME padding preserves input feature map size if uninary strides

'''
3 things are happening here:
1. Flattens the filter or layer1_weights to a 2D matrix with shape [filter_height*filter_width*in_channels, out_channels], or [5*5*1,16] or [25,16]
2. Exacts image patches from input tensor/feature map to form a virual/temp tensor of shape [batch, out_height, out_weight, filter_height*filter_width*in_channels] or [16,28/2,28/2,5*5*1] or [16,14,14,25]
3. Multiplies image patch of shape [16,14,14,25] with flattened filter [25,16]
4. Result (ConvLayer1) should be of shape [16,14,14,16]
'''

print (("Kernal 1: %s")%(layer1_weights.get_shape()))
print (("ConvLayer 1: %s")%conv.get_shape())
hidden = tf.nn.relu(conv + layer1_biases) # okay: tensor.shape==[16,14,14,16] + tensor.shape==[16] or tensor.shape==[14,16] or tensor.shape==[14,14,16]
print (("Biases 1: %s")%layer1_biases.get_shape())
print (("Hidden 1: %s")%hidden.get_shape()

In [14]:
conv = tf.nn.conv2d(train_dataset[0:16,], layer1_weights, strides = [1, 1, 1, 1], padding='SAME') # [16,28,28,1], [5,5,1,16] = [16,28,28,16]
hidden = tf.nn.relu(conv + layer1_biases) # [16,28,28,16]
print(hidden.get_shape())
pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # [16,14,14,16]
print(pool.get_shape())

In [15]:
# Variables.
layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], # [filter_height, filter_width, in_channel, out_channel] or [5,5,16,16]
                                                 stddev=0.1)) # why is stddev = 0.1?
layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth])) # to avoid dead neurons

# Convolution Starts.
conv2 = tf.nn.conv2d(hidden, layer2_weights, strides = [1,2,2,1], padding='SAME') 
hidden2 = tf.nn.relu(conv2 + layer2_biases)

'''
1. flattens layer2_weights from [5,5,16,16] to [5*5*16,16] or [400,16]
2. images patches from conv1 [16,14,14,16] to [16,14/2,14/2,5*5*16] or [16,7,7,400]
3. [16,7,7,400] multiplies [400,16]: [16,7,7,16]
'''

print(hidden2.get_shape())

In [16]:
conv2 = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME') # [16,14,14,16], [5,5,16,16] = [16,14,14,16]
hidden2 = tf.nn.relu(conv2 + layer2_biases) # [16,14,14,16]
print(hidden2.get_shape())
pool2 = tf.nn.max_pool(hidden2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #[16,7,7,16]
print(pool2.get_shape())

In [17]:
# Variables.
layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], # [28//4*28//4*16,64] or [7,7,64]
                                                 stddev=0.1)) # for use in the fully connected layer
layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

shape = hidden2.get_shape().as_list()
print(shape)
reshape = tf.reshape(hidden2, [shape[0], shape[1] * shape[2] * shape[3]])
print(reshape.get_shape())

hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) # [16,7*7*16] multiplies [28//4*28//4*16,64], resulting in [16,64]
print(hidden3.get_shape())

In [18]:
layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], # [28//4*28//4*16,64] or [7,7,64]
                                                 stddev=0.1)) # for use in the fully connected layer
layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

shape = pool2.get_shape().as_list() #[16,7,7,16]
reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]]) #[16,784]
hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) #[16,64], this is the fully connected layer

y_conv = tf.nn.softmax(tf.matmul(tf.nn.dropout(hidden3, 0.5), W_fc2) + b_fc2) # implements dropout

In [19]:
layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1)) # [64,10]
layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels])) # [10]

logits = tf.matmul(hidden3, layer4_weights) + layer4_biases # [16,64] * [64,10] + [10], resulting in [16,10]
print(logits.get_shape())

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
print(loss)

### Problem 1

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation `(nn.max_pool())` of stride 2 and kernel size 2.

In [21]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels)) #num_channels = 1 is grayscale
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1)) #[5,5,1,16]
  layer1_biases = tf.Variable(tf.zeros([depth])) #[16]
  
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1)) #[5,5,16,16]
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth])) #[16]
  
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1)) #[784,64]
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden])) #[64]
  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1)) #[64,10]
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels])) #[64,10]
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, strides = [1, 1, 1, 1], padding='SAME') # [16,28,28,1], [5,5,1,16] = [16,28,28,16]
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # [16,14,14,16]
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME') # [16,14,14,16], [5,5,16,16] = [16,14,14,16]
    hidden = tf.nn.relu(conv + layer2_biases) # [16,14,14,16]
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #[16,7,7,16]
    
    shape = pool.get_shape().as_list() #[16,7,7,16]
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]]) #[16,784]
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) #[16,64]
    
    return tf.matmul(hidden, layer4_weights) + layer4_biases #[16,10]
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [22]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
      
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

### Problem 2

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

In [24]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels)) #num_channels = 1 is grayscale
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1)) #[5,5,1,16]
  layer1_biases = tf.Variable(tf.zeros([depth])) #[16]
  
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1)) #[5,5,16,16]
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth])) #[16]
  
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1)) #[784,64]
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden])) #[64]
  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1)) #[64,10]
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels])) #[64,10]
  
  # Model.
  def model(data, dropout = False):
    if dropout == True: 
      keep_prob = 0.5
    else:
      keep_prob = 1
      
    conv = tf.nn.conv2d(data, layer1_weights, strides = [1, 1, 1, 1], padding='SAME') # [16,28,28,1], [5,5,1,16] = [16,28,28,16]
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # [16,14,14,16]
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME') # [16,14,14,16], [5,5,16,16] = [16,14,14,16]
    hidden = tf.nn.relu(conv + layer2_biases) # [16,14,14,16]
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #[16,7,7,16]
    
    shape = pool.get_shape().as_list() #[16,7,7,16]
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]]) #[16,784]
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) #[16,64], this is the fully connected layer

    return tf.matmul(tf.nn.dropout(hidden, keep_prob), layer4_weights) + layer4_biases #[16,10], implements dropout
  
  # Training computation.
  logits = model(tf_train_dataset, dropout = True)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
#   # Optimizer.
#   global_step = tf.Variable(0)  # count the number of steps taken.
#   learning_rate = tf.train.exponential_decay(0.05, global_step, decay_steps = 3000, decay_rate = 0.5)
#   # Passing global_step to minimize() will increment it at each step.
#   optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step) 
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, dropout = False)) # shouldn't use dropout for calculating accuracy
  test_prediction = tf.nn.softmax(model(tf_test_dataset, dropout = False))

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
      
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

In [26]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels)) #num_channels = 1 is grayscale
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1)) #[5,5,1,16]
  layer1_biases = tf.Variable(tf.zeros([depth])) #[16]
  
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1)) #[5,5,16,16]
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth])) #[16]
  
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1)) #[784,64]
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden])) #[64]
  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1)) #[64,10]
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels])) #[64,10]
  
  # Model.
  def model(data, dropout = False):
    if dropout == True: 
      keep_prob = 0.5
    else:
      keep_prob = 1
      
    conv = tf.nn.conv2d(data, layer1_weights, strides = [1, 1, 1, 1], padding='SAME') # [16,28,28,1], [5,5,1,16] = [16,28,28,16]
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # [16,14,14,16]
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME') # [16,14,14,16], [5,5,16,16] = [16,14,14,16]
    hidden = tf.nn.relu(conv + layer2_biases) # [16,14,14,16]
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #[16,7,7,16]
    
    shape = pool.get_shape().as_list() #[16,7,7,16]
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]]) #[16,784]
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) #[16,64], this is the fully connected layer

    return tf.matmul(tf.nn.dropout(hidden, keep_prob), layer4_weights) + layer4_biases #[16,10], implements dropout
  
  # Training computation.
  logits = model(tf_train_dataset, dropout = True)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, decay_steps = 3000, decay_rate = 0.5)
  # Passing global_step to minimize() will increment it at each step.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step) 
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, dropout = False)) # shouldn't use dropout for calculating accuracy
  test_prediction = tf.nn.softmax(model(tf_test_dataset, dropout = False))

In [27]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
      
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

In [28]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden1 = 512
num_hidden2 = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels)) #num_channels = 1 is grayscale
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1)) #[5,5,1,16]
  layer1_biases = tf.Variable(tf.zeros([depth])) #[16]
  
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1)) #[5,5,16,16]
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth])) #[16]
  
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden1], stddev=0.1)) #[784,128]
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1])) #[128]
  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden1, num_hidden2], stddev=0.1)) #[128,64]
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2])) #[64]
  
  layer5_weights = tf.Variable(tf.truncated_normal([num_hidden2, num_labels], stddev=0.1)) #[64,10]
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels])) #[10]

  # Model.
  def model(data, dropout = False):
    if dropout == True: 
      keep_prob = 0.8
    else:
      keep_prob = 1
      
    conv = tf.nn.conv2d(data, layer1_weights, strides = [1, 1, 1, 1], padding='SAME') # [16,28,28,1], [5,5,1,16] = [16,28,28,16]
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') # [16,14,14,16], max_pooling
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME') # [16,14,14,16], [5,5,16,16] = [16,14,14,16]
    hidden = tf.nn.relu(conv + layer2_biases) # [16,14,14,16]
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #[16,7,7,16]
    
    shape = pool.get_shape().as_list() #[16,7,7,16]
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]]) #[16,784]
    hidden1 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) #[16,128], this is the first fully connected layer

    hidden2 = tf.matmul(tf.nn.dropout(hidden1, keep_prob), layer4_weights) + layer4_biases # dropout, [16,128],[128,64] 
    
    return tf.matmul(tf.nn.dropout(hidden2, keep_prob), layer5_weights) + layer5_biases #[16,10], implements dropout
  
  # Training computation.
  logits = model(tf_train_dataset, dropout = True)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, decay_steps = 3000, decay_rate = 0.5)
  # Passing global_step to minimize() will increment it at each step.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step) 
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, dropout = False)) # shouldn't use dropout for calculating accuracy
  test_prediction = tf.nn.softmax(model(tf_test_dataset, dropout = False))

In [29]:
num_steps = 3501

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
      
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

General observations:
1. Hard to pick learning rate when using learning rate exponential decay
2. Every time I double `num_hidden`, I get a 0.2% accuracy improvement.